In [1]:
from implicit.als import AlternatingLeastSquares as ALS
from scipy import sparse

import json
import os
import pandas as pd
import numpy as np

In [2]:
%%time
FILE_PATH = '../data/'

train = pd.read_json(FILE_PATH+'train.json', typ = 'frame')
test = pd.read_json(FILE_PATH+'test.json', typ = 'frame')
val = pd.read_json(FILE_PATH+'val.json', typ = 'frame')

CPU times: user 1.56 s, sys: 140 ms, total: 1.7 s
Wall time: 1.7 s


In [3]:
%%time

row, col, data = [], [], []

for idx, playlist in enumerate(train['songs'].to_numpy()):
    row += [idx]*len(playlist)
    col += playlist
    data += [1]*len(playlist)

data = sparse.csr_matrix((data, (row, col)))

#playlists-songs matrix 생성

CPU times: user 2.13 s, sys: 78.9 ms, total: 2.21 s
Wall time: 2.21 s


In [4]:
wrmf = ALS(factors=200, regularization=0.001)
wrmf.fit(data.T*100)

In [5]:
import torch
import random
from torch.utils.data import Dataset

class CNN_Dataset(Dataset):
    def __init__(self, data, als_model):
        self.input_data = data
        self.als_model = als_model
        self.songs_embeddings = als_model.item_factors
        
    def __len__(self):
        return self.input_data.shape[0]
    
    def __getitem__(self, idx):
        rec = self.als_model.recommend(idx, self.input_data, N=1000)
        idx_list = [idx for idx, _ in rec]
        negative_list = [i for i in range(self.input_data.shape[1]) if i not in idx_list]
        negative_list = random.sample(negative_list, 200)
        data = torch.FloatTensor(np.reshape(self.songs_embeddings[idx_list], (200, -1, 1)))
        negative = torch.FloatTensor(np.reshape(self.songs_embeddings[negative_list], (200, -1, 1)))
        return data, negative
    
#Loss텀을 위해서 후보군 1000개와 후보군에 속하지 않은 곡 200개를 뽑음

In [6]:
%%time

dataset=CNN_Dataset(data, wrmf)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs


In [7]:
from torch.utils.data import DataLoader

trainloader = torch.utils.data.DataLoader(dataset,batch_size=1,shuffle=True, drop_last=True)

In [8]:
%%time

candidate, negative = next(iter(trainloader))
print("후보군 : ", candidate.shape)
print("후보군이 아닌 것 : ", negative.shape)

후보군 :  torch.Size([1, 200, 1000, 1])
후보군이 아닌 것 :  torch.Size([1, 200, 200, 1])
CPU times: user 1min 52s, sys: 477 ms, total: 1min 52s
Wall time: 1min 49s


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as weight_init
from torch.autograd import Variable

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('use: ',device)

class GLU(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GLU, self).__init__()
        self.conv = nn.Conv2d(
            in_channels=in_channels, 
            out_channels=out_channels*2,
            kernel_size=(3,1),
            padding=(1,0)
        )
        
    def forward(self, x):
        out = self.conv(x)
        out, gate = out.split(int(out.shape[1] / 2), 1)
        out = out * torch.sigmoid(gate)
        return out
        
        
class CNN(nn.Module):
    def __init__(self, layer_size=7, embedding_size=200, output_size=900):
    #layer_sizes : a number of GLU block 
        super(CNN, self).__init__()
        self.GLU = self._make_layer(layer_size, embedding_size, output_size)
        self.FCL = nn.Linear(output_size*3, 200)
    
    def _make_layer(self, layer_size, embedding_size, output_size):
        layers = []
        for i in range(layer_size-1):
            if i==0:
                layers.append(GLU(
                    in_channels=embedding_size, 
                    out_channels=output_size
                ))
            else:
                layers.append(GLU(
                    in_channels=output_size, 
                    out_channels=output_size
                ))
            layers.append(nn.BatchNorm2d(output_size))
            layers.append(nn.ReLU())
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.GLU(x)
        out = torch.topk(out, k=3, dim=2)[0]
        out = torch.flatten(out, start_dim=1)
        out = self.FCL(out)
        out = torch.unsqueeze(out, 1)
        return out

use:  cuda


In [23]:
model=CNN(7, 200, 900)
model.to(device)

CNN(
  (GLU): Sequential(
    (0): GLU(
      (conv): Conv2d(200, 1800, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
    )
    (1): BatchNorm2d(900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): GLU(
      (conv): Conv2d(900, 1800, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
    )
    (4): BatchNorm2d(900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): GLU(
      (conv): Conv2d(900, 1800, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
    )
    (7): BatchNorm2d(900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): GLU(
      (conv): Conv2d(900, 1800, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
    )
    (10): BatchNorm2d(900, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): GLU(
      (conv): Conv2d(900, 1800, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
    )
    (13): BatchNorm2d(900, eps=1

In [38]:
def loss_fn(data, negative, pred, k):
    out1 = torch.bmm(pred,torch.squeeze(data, -1)[:,:,:k])
    out1 = torch.log(1/(torch.exp(-out1)+1))
    out1 = -torch.sum(out1)
    out2 = torch.bmm(pred,torch.squeeze(negative, -1))
    out2 = torch.log(1-1/(torch.exp(-out2)+1))
    out2 = -torch.sum(out2) 
    return (out1+out2)/data.shape[0]

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=5e-4)

In [29]:
from tqdm import trange

def train(model, trainloader, optimizer, loss_fn, k=200, n_epoch=10):
    model.train()
    pbar = trange(n_epoch, desc='Loss : 0', leave=True, position=0)
    for epoch in pbar:
        for X, Y in trainloader:
            X = X.to(device)
            Y = Y.to(device)
            pred = model(X) 
            loss = loss_fn(X, Y, pred, k)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.set_description("Loss : %.3f" % loss)
    return loss_list

In [ ]:
train(model, trainloader, optimizer, loss_fn, 200, 1)

Loss : 269.689:   0%|          | 0/1 [1:29:22<?, ?it/s]